Imports des bibliothèques

In [25]:
from googleapiclient.discovery import build
import json

API keys credentials

In [26]:
#read credentials
f = open('credentials.json')
data = json.load(f)
f.close()

api_key = data['api_key']
youtube = build('youtube', 'v3', developerKey = api_key)

Recherche youtube

In [27]:
request = youtube.search().list(
        part="snippet",
        maxResults=50,
        order="viewCount",
        q="chatgpt"
    )
response = request.execute()

Stockage des identifiants des vidéos

In [28]:
list_videos_id = []
result_video_length = len(response['items'])

for i in range(result_video_length):
    list_videos_id.append(response['items'][i]['id']['videoId'])

print(list_videos_id)

['jPhJbKBuNnA', 'Sqa8Zo2XWc4', 'JTxsNm9IdYU', 'kh5dN72GTQ8', '7uljoeHgN4g', '0uQqMxXoNVs', 'sTeoEFzVNSc', 'rSCNW1OCk_M', '8I3NTE4cn5s', 'Nh5i5obZ8OY', 'WPBWIcprLxc', 'kuTTAuUorsI', 'iWhlrkfJrCQ', 'WizoCwjEKsg', 'JYtZ2zsdE_s', 'K7z0jlE61IM', 'gtgA4u8V_TQ', '_eHjifELI-k', 'wnNp-zT3JmY', 'ng438SIXyW4', 'AXn2XVLf7d0', 'BYGWMArFNRE', 'pOmpqdlVCoo', 'MIdv9ku1TfU', 'kPV4BgmoXz8', 'RCBZ-Xj2dgQ', 'QfrF_rwihyA', 'ey1kHYVu19c', 'kQPUWryXwag', 'AggvJLRqgDg', 'YNEb3EK4rNY', 'CtNsdVtQsf0', 'X6MykhFaP5U', 'Pi-5_eid7VA', 'pc8ftlzB2D0', 'Ku-I7KVmqdw', 'lGSGTrn5INA', 'UNbyT7wPwk4', '1EeYEkrlD8k', 'Fw5ybNwwSbg', '-WSQEyT2Mu0', 'Hrv4UP3wBRg', '40Kp_fa8vIw', 'qLh1GIPBXhc', '5MYJtKx-ebE', 'o5MutYFWsM8', 'y3iLOxBTuy4', '3vr3wcCAZbY', 'W3sk8gPpmcc', 'L6vW0um5XTg']


Génération des commentaires youtube de la première vidéo de la liste list_video_id

In [29]:
request_comments = youtube.commentThreads().list(
        part="snippet, replies",
        order="relevance",
        videoId= list_videos_id[0], 
        )
response_comments = request_comments.execute()

Récupération des données des commentaires et de l'utilisateur

In [30]:
videoId = response_comments['items'][0]['snippet']['topLevelComment']['snippet']['videoId']
comment = response_comments['items'][0]['snippet']['topLevelComment']['snippet']['textOriginal']
author = response_comments['items'][0]['snippet']['topLevelComment']['snippet']['authorDisplayName']
authorChannelUrl = response_comments['items'][0]['snippet']['topLevelComment']['snippet']['authorChannelUrl']
likeCount = response_comments['items'][0]['snippet']['topLevelComment']['snippet']['likeCount']
publishedAt = response_comments['items'][0]['snippet']['topLevelComment']['snippet']['publishedAt']

Récupération des réponses aux commentaires et des données utilisateurs

In [31]:
replies_response_snippet = response_comments['items'][0]['replies']['comments'][0]['snippet']
replies_videoId = replies_response_snippet['videoId']
replies_comment = replies_response_snippet['textOriginal']
replies_author = replies_response_snippet['authorDisplayName']
replies_authorChannelUrl = replies_response_snippet['authorChannelUrl']
replies_likeCount = replies_response_snippet['likeCount']
replies_publishedAt = replies_response_snippet['publishedAt']

Initialisation des listes

In [32]:
videoId = []
comment = []
author = []
authorChannelUrl = []
likeCount = []
publishedAt = []
replies_videoId = []
replies_comment = []
replies_author = []
replies_authorChannelUrl = []
replies_likeCount = []
replies_publishedAt = []

Récupération des données

In [33]:
for id in list_videos_id:
    request_comments = youtube.commentThreads().list(
        part="snippet, replies",
        order="relevance",
        videoId= id, 
        )
    response_comments = request_comments.execute()

    nextPageToken = 0
    for i in range(5):
        nextPageToken = response_comments['nextPageToken']
        response_count = len(response_comments['items'])
        for j in range(response_count):
            response_snippet = response_comments['items'][j]['snippet']['topLevelComment']['snippet']
            videoId.append(response_snippet['videoId'])
            comment.append(response_snippet['textOriginal'])
            author.append(response_snippet['authorDisplayName'])
            authorChannelUrl.append(response_snippet['authorChannelUrl'])
            likeCount.append(response_snippet['likeCount'])
            publishedAt.append(response_snippet['publishedAt'])
            if "replies" in response_comments['items'][j]:
                number_replies = len(response_comments['items'][j]['replies']['comments'])
                for k in range(number_replies):
                    replies_response_snippet = response_comments['items'][j]['replies']['comments'][k]['snippet']
                    replies_videoId.append(replies_response_snippet['videoId'])
                    replies_comment.append(replies_response_snippet['textOriginal'])
                    replies_author.append(replies_response_snippet['authorDisplayName'])
                    replies_authorChannelUrl.append(replies_response_snippet['authorChannelUrl'])
                    replies_likeCount.append(replies_response_snippet['likeCount'])
                    replies_publishedAt.append(replies_response_snippet['publishedAt'])
        request_comments = youtube.commentThreads().list(part="snippet, replies", order="relevance", 
                                                         videoId=id, pageToken= nextPageToken)
        response_comments = request_comments.execute()

Création d'un DataFrame pour le stockage de données

In [34]:
import pandas as pd

columns=['videoId','comment','author', 'authorChannelUrl', 'likeCount', 'publishedAt', 'replies_videoId', 'replies_comment',
                            'replies_author', 'replies_authorChannelUrl', 'replies_likeCount', 'replies_publishedAt']
df = pd.DataFrame(list(zip(videoId, comment, author, authorChannelUrl, likeCount, publishedAt, replies_videoId, replies_comment,
                            replies_author, replies_authorChannelUrl, replies_likeCount, replies_publishedAt)),
                 columns=columns )
print(df)

          videoId                                            comment  \
0     jPhJbKBuNnA  It's an opinion piece this week! Been a while ...   
1     jPhJbKBuNnA  “I've come up with a set of rules that describ...   
2     jPhJbKBuNnA  I was genuinely expecting and terrified that t...   
3     jPhJbKBuNnA  “Surprise is just a fancy word for being wrong...   
4     jPhJbKBuNnA  My aunt worked for AT&T in the 80s.  I remembe...   
...           ...                                                ...   
4993  L6vW0um5XTg  This is awesome. And terrifying. I don’t want ...   
4994  L6vW0um5XTg  I think you should do a race between two ppl, ...   
4995  L6vW0um5XTg  You can have it write scripts in sonic pi, so ...   
4996  L6vW0um5XTg  In the end I can prototype that ridiculous ide...   
4997  L6vW0um5XTg  I discovered that if ChatGPT stops mid sentenc...   

                author                                   authorChannelUrl  \
0            Tom Scott  http://www.youtube.com/channel/UCB

Sauvegarde de la DataFrame dans un fichier excel

In [35]:
df.to_excel(r'D:\maxim\Documents\Projects\CollecteDesDonnees\Youtube\YoutubeRaw.xlsx', index=False)

Sauvegarde de la DataFrame dans un fichier JSON

In [36]:
out = df.to_json(orient='records')[1:-1].replace('},{', '} {')

with open('youtube_data.json', 'w') as f:
    f.write(out)